In [42]:
!ls dodfs/results/

txt  txt.zip


In [43]:
import glob
import re
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import unicodedata as uni
from collections import Counter
import gc

In [44]:
df_validado = pd.read_csv('TCDF_data/Atos_Aposentadoria_validados.csv',
                         index_col=False)
df_validado.shape

(5516, 19)

In [45]:
def get_num(s):
    return int(re.search('\d+', s).group())

def get_date(s):
    return re.search('\d+[-]\d+[-]\d+', s).group()
def get_rdate(s):
    return '-'.join(re.search('\d+[-]\d+[-]\d+', s).group().split('-'))

def erase(p, s):
    return re.sub(p, '', s)

data = pd.read_csv("verif_dataV1.csv")
data['labels'] = ""
data.rename(mapper=lambda x: erase('_dodf', x.lower()),
            axis=1,inplace=True)

In [46]:
data.columns

Index(['ref_anomes', 'data', 'num', 'pagina', 'tipo', 'ato', 'cod_empresa',
       'empresa_ato', 'cod_matricula_ato', 'cod_matricula_sigrh', 'cpf',
       'nome_ato', 'nome_sigrh', 'cargo', 'classe', 'padrao', 'quadro',
       'processo', 'fund_legal', 'text', 'labels'],
      dtype='object')

In [47]:
cols = ['data', 'num', 'pagina', 'tipo', 'nome_ato', 'quadro']
df = data[cols].copy()
df.rename(
    {'pagina':'pag',
    'nome_ato':'nome'
    },
axis=1,inplace=True)
df['pag'] = df.pag.str.replace('[^\d]+', '').astype(np.int16)
df.sort_values(['data', 'num', 'pag', 'nome'], inplace=True)
df.to_csv('quadro.csv', index=False)

In [48]:
df.quadro.value_counts()

QUADRO DE PESSOAL DO DISTRITO FEDERAL                                           3285
QUADRO DE PESSOAL DA SECRETARIA DE ESTADO DE SAUDE DO DISTRITO FEDERAL          1164
QUADRO SUPLEMENTAR                                                               138
QUADRO DE PESSOAL DO SERVICO DE LIMPEZA URBANA DO DISTRITO FEDERAL               127
QUADRO SUPLEMENTAR DO DISTRITO FEDERAL                                            71
QUADRO DE PESSOAL DO GOVERNO DO DISTRITO FEDERAL                                  64
QUADRO DE PESSOAL DO DEPARTAMENTO DE ESTRADAS DE RODAGEM DO DISTRITO FEDERAL      45
QUADRO DE PESSOAL DO SERVICO DE LIMPEZA URBANA                                    39
QUADRO DE PESSOAL DO DEPARTAMENTO DE TRANSITO DO DISTRITO FEDERAL                 25
QUADRO DE PESSOAL DOS SERVICOS AUXILIARES DESTE TRIBUNAL                          21
QUADRO DE PESSOAL DA AGENCIA DE FISCALIZACAO DO DISTRITO FEDERAL                  20
QUADRO DE PESSOAL DOS SERVICOS AUXILIARES DESTA CORTE            

In [49]:
%%time
lis = [open(i, 'r') for i in glob.glob('dodfs_flat/*.txt')]
txt = {i.name.split('/')[-1]: i.read() for i in lis}
txt = {(get_rdate(k), get_num(k), k): v for (k,v) in txt.items()}

CPU times: user 347 ms, sys: 457 ms, total: 804 ms
Wall time: 1.71 s


In [50]:
def cand_files(date, num):
    padded = '0'*( num < 100 + num < 10) + str(num)
    date = date.replace('/', '-')
    cands = [i for i in glob.glob('dodfs_flat/*.txt') if date in i and padded in i]
    return cands

def preprocess(s):
    s=uni.normalize('NFKD', s).encode(
        'ascii', errors='ignore').decode()
    s = re.sub('-\n', '-', s)
    s = re.sub('\n', ' ', s)
    s = re.sub('\s{2,}', ' ', s)
    return s

def reg_nome_quadro(nome, quadro):
    start = r'(?im:'
    nome = '[-]?\s+?'.join(list(nome))
    nome = nome.replace(' ', '\s+')
    body = nome + '(.){0,500}?' + quadro.replace(' ','\s+')
    end = ')'
    return start + body + end

In [51]:
dfc = df[df.quadro.notna()].copy()
dfc['data'] = dfc.data.str.replace('/', '-')
dfc.head(1)

,data,num,pag,tipo,nome,quadro
45,01-01-2019,248,13,NORMAL,ADERALDO LUIZ DA SILVA,QUADRO DE PESSOAL DO DISTRITO FEDERAL


In [52]:
%%time
cols = ['data', 'num']
_ = dfc[cols].copy()
_.set_index(cols, inplace=True)
cached_fnames = {i:cand_files(*i) for i in _.index}
# ` $$CAFEBABE$$ ` é um separador entre os textos de dois textos pois
# alguns DODFs compartilham número e data, mas diferem no tipo de edição

cached_text = {k: preprocess(' $$CAFEBABE$$ '.join([open(jj, 'r').read() for jj in val]))
                   for (k, val) in cached_fnames.items()}
del _
gc.collect()

CPU times: user 13.4 s, sys: 2.08 s, total: 15.5 s
Wall time: 15.5 s


2172

In [53]:
%%time
dfc['match'] = dfc.apply(
    lambda x: re.search(reg_nome_quadro(x.nome, x.quadro), cached_text[x.data, x.num],
                        flags=re.MULTILINE|re.DOTALL), axis=1
)
dfc['match'].head()

CPU times: user 53.1 s, sys: 28.8 ms, total: 53.1 s
Wall time: 53.1 s


45    None
13    None
11    None
40    None
24    None
Name: match, dtype: object

In [54]:
%%time
dfc['match'] = dfc.match.map(
    lambda x: x.group(),
    na_action='ignore'
)

CPU times: user 2.08 ms, sys: 0 ns, total: 2.08 ms
Wall time: 1.75 ms


In [55]:
dfc.head(2)

,data,num,pag,tipo,nome,quadro,match
45,01-01-2019,248,13,NORMAL,ADERALDO LUIZ DA SILVA,QUADRO DE PESSOAL DO DISTRITO FEDERAL,None
13,01-01-2019,248,13,NORMAL,ALEXANDRE ZILAHI JUNIOR,QUADRO DE PESSOAL DO DISTRITO FEDERAL,None


In [56]:
dfc.drop(['tipo'], axis=1).to_csv('quadro-match.csv', index=False)
dfc[dfc.match.isna()].drop(['tipo'], axis=1).to_csv('quadro-missing.csv', index=False)
dfc[dfc.match.notna()].drop(['tipo'], axis=1).to_csv('quadro-found.csv', index=False)

# Após análise manual dos casos falhos, e de inspeção da planilhados casos bem sucedidos, constatou-se que todos os quadros estão corretos

In [57]:
dfc['correto'] = 1
dfc.drop('match', axis=1, inplace=True)
dfc.to_csv('quadro-final.csv', index=False)